# Use OpenAI API to Analyze Your Product Reviews in a New Way

In this notebook, we will explore how to use OpenAI API to write prompts and build a chatbot that can analyze customer insights in a very new way. **The aim is to help product managers with their discovery work.**

For our use case, we will focus on the Airbnb app and use the [reviews from the Apple App Store](https://apps.apple.com/us/app/airbnb/id401626263?see-all=reviews) as our data source.

The comments on the code are detailed to enable everyone to find their way around, even beginners.

This notebook is inspired by and adapted from the [DeepLearning.AI course ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/1/introduction), which I highly recommend.

Interested in learning more about AI for product management?
Check out my newsletter [AI for Product](https://www.linkedin.com/newsletters/7152255424174387200/).

## 1. Install and import the required libraries

In [1]:
# Install the required libraries
!pip install requests
!pip install beautifulsoup4
!pip install openai

In [2]:
# Import the required libraries
import requests #send HTTP requests to web servers and get the data from them
from bs4 import BeautifulSoup #parse and extract information from HTML documents
import openai #access the OpenAI API

from dotenv import load_dotenv, find_dotenv #aim to store sensitive information securely
_ = load_dotenv(find_dotenv())

from IPython.display import display, HTML #advanced features such as displaying HTML content

## 2. Extract Airbnb reviews from the App Store

Let's scrape the reviews of the [Airbnb app from the App Store website](https://apps.apple.com/us/app/airbnb/id401626263?see-all=reviews) and store them in a list.

In [3]:
# Define the URL of the product reviews
url = "https://apps.apple.com/us/app/airbnb/id401626263?see-all=reviews"

In [4]:
# Make a request to the URL and parse the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

It took me a while to inspect the web page and find the class that lets you scrap the review.
You can inspect the web page to see why I finally chose this class.

In [5]:
# Find all the elements that contain the review texts
reviews = soup.find_all("div", class_="l-column--grid l-column small-12 medium-6 large-4 small-valign-top l-column--equal-height")

In [6]:
# Create an empty list to store the reviews
review_list = []

In [7]:
# Loop through each review in the reviews variable
for review in reviews:
    # Find the element that contains the review title and get its content
    title = review.find("h3", class_="we-truncate we-truncate--single-line we-customer-review__title").get_text()
    
    # Find the element that contains the review text and get its content
    text = review.find("blockquote", class_="we-truncate we-truncate--multi-line we-truncate--interactive we-customer-review__body").get_text()
    
    # Append the title and text as a tuple to the review list
    review_list.append((title, text))

Now, we have a list of the reviews, each with a title and description. Let's check that we have ten reviews and can call one of them.

In [8]:
len(review_list) # Must give 10

10

In [9]:
# Let's look at one of the reviews
print(review_list[5])

('\n    If your are under 25 don’t open an account\n', '\nI am from Connecticut and I was in Austin Texas for a work related trip. I specifically listed that it was a work related trip paid for by work. However, I was booking AirBnB on my personal account. I am 21 years old, and according to AirBnB they won’t allow and restrict my account so I cannot book an entire place for myself and I am forced to only rent rooms in peoples houses. This is direct discrimination based on my age it believes I am “high risk” to throw a party in an airBnb in a place I know literally no one. This is an absolute joke and I plan on deleting my AirBnB account since costumer service, which btw is out sourced to India so AirBnB does not support American jobs, told me there is nothing I can do but book shared rooms or private rooms in homes with the owners on site. This is obviously an inconvenience to me because I am a business professional spending an extended amount of time away from home and not all hosts 

## 3. Improve your prompting

To start with, we're going to explore these reviews by applying some best practices in prompting.

### 3.1. Set the API key and define the helper function

Get your OpenAI API key from your OpenAI account ([link](https://platform.openai.com/api-keys)), copy and past it below.

In [10]:
# Set your OpenAI API key
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxC9F"

Define get_completion, a helper function that takes a prompt as an input and returns a response as an output.

In [11]:
# Define the helper function to use prompts and look at the generate output
client = openai.OpenAI(api_key=openai.api_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

If you are curious, in this helper function:
- client: represents the connection to the OpenAI API service.
- prompt: string that contains the text as an input.
- model: determines the quality and style of the output.
- messages: represents the prompt as a message.
- response: contains the data that the OpenAI API returns.
- temperature: controls the creativity of the output. A lower temperature means more predictable and consistent output. A higher temperature means more diverse and surprising output.

### 3.2. Use principles and tactics for effective prompting

Principle 1: Be clear and specific.

Principle 2: Give time to the model to think.

#### 3.2.1. Principle 1: Be clear and specific

**Tactic 1: use delimiters.**

Without delimiters, the model runs the risk of interpreting a review as additional actions to complete. To avoid this, delimiters help the model correctly interpret what is expected.

Possible delimiters:
- Triple quotes: """
- Triple backticks: '''
- Triple dashes: ---
- Angle brackets: < >
- XML tags: `<tag> </tag>`

Imagine we want to have a one-line summary of the first review.

In [12]:
prompt = f"""
Summarize the text delimited by triple backticks
into a single sentence.
```{review_list[0]}```
"""
response = get_completion(prompt)
print(response)

The user loves the app as a host but feels that it is missing a key feature, which is the ability to manually check a guest out of their unit in order to leave a review and accommodate new daily guests, as well as the ability to turn off notifications for upcoming guest arrivals and utilize IF functions for shortcode messages to avoid potential problems.


**Tactic 2: ask for structured output.**

As you'd expect, telling the model the output format helps it to be precise and consistent.

Still on the first review, imagine we'd like to turn it into a list of pros and cons and get a JSON output. Just say the word.

In [13]:
prompt = f"""
Generate a list of pros and cons
using the review in brackets. 
Provide them in JSON format with the following keys: 
title, pros, cons.
```{review_list[0]}```
"""
response = get_completion(prompt)
print(response)

{
  "title": "Love the app as a host, but is missing one key feature for hosts",
  "pros": [
    "The app is amazing",
    "Expanded ability to include personal info in scheduled messages"
  ],
  "cons": [
    "Inability to manually check a guest out and leave a review",
    "Waiting until 12-3 pm to leave a review",
    "Inability to turn off notifications for guest arrivals",
    "Lack of IF functions for shortcode messages",
    "Difficulty in handling missing guest information in shortcode messages"
  ]
}


**Tactic 3: ask the model to check whether conditions are satisfied.**

As you don't always know what the model will fall on in your data, it can be useful to tell the model what to do if it finds something odd.

Let's take the example of the 3rd review. The user shares his bad adventures with Airbnb app. We'll ask the model to transform the review into a sequence of steps. This can be very useful for starting a user journey.

We'll also ask the model to tell us if it doesn't succeed in case we use the prompt on another review not relevant for this task.

In [14]:
prompt = f"""
You will be provided with a review delimited by triple quotes. 
If it contains a sequence of a customer experience,
re-write the steps of the user journey in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{review_list[2]}\"\"\"
"""
response = get_completion(prompt)
print(response)

Step 1 - The customer rented a place for a family vacation based on the photos.
Step 2 - Upon arrival, the customer found that the place was not clean.
Step 3 - The customer noticed dust everywhere, dirty floors, and a sticky kitchen.
Step 4 - The customer also found a broken curtain and hair from previous tenants in the beds.
Step 5 - The worst discovery was finding mold in the bathroom.
Step 6 - The customer realized that the vacation was ruined and couldn't stay there.
Step 7 - The customer requested a refund for the second and last day of stay.
Step 8 - The host refused to refund and claimed they would fix everything.
Step 9 - The customer didn't want strangers cleaning during their family vacation.
Step 10 - The customer received only $34 back out of the $1200 spent.
Step 11 - The customer blames both the host and Airbnb for the poor service.
Step 12 - The customer had to rent a hotel for $700 to avoid staying on the street.
Step 13 - Overall, the customer lost $1200 and had to sp

#### 3.2.2. Principle 2: Give time to the model to think

**Tactic 1: specify the steps to complete a task.**

To help the model complete a task, we can break it down into smaller and simpler steps.

Let's imagine that the 4th review is of particular interest to us. Rather than asking the model to immediately draw up a list of pain points and possible solutions we're going to suggest that it proceed in stages and think about priorization.

In [15]:
prompt = f"""
Perform the following actions: 
1 - List the user's pain points expressed in the review
delimited by triple backticks.
2 - Prioritize them by criticality.
3 - Suggest solutions for each pain point.
4 - Output a JSON object that contains the following
keys: pain_point, possible_solutions.

Separate your answers with line breaks.

Text:
```{review_list[3]}```
"""
response = get_completion(prompt)
print(response)

1 - The user's pain points expressed in the review are:
- Booking was cancelled by the host at the last minute.
- Misleading information about the rental (thought it was a whole apartment with 2 bedrooms, but turned out to be a one bedroom booking sharing with other visitors).
- Had to book another rental with extra costs.
- Paid a penalty of 70% of the total for cancelling the booking.
- Disappointing customer service from Airbnb.
- Lack of assistance or resolution from Airbnb customer service.
- Feeling blamed by Airbnb customer service for not paying attention to details.
- Feeling that Airbnb allows hosts to cancel bookings without penalties.

2 - Prioritizing the pain points by criticality:
1. Booking was cancelled by the host at the last minute.
2. Misleading information about the rental.
3. Disappointing customer service from Airbnb.
4. Had to book another rental with extra costs.
5. Paid a penalty of 70% of the total for cancelling the booking.
6. Lack of assistance or resoluti

**Tactic 2: specify the format of the output.**

With the same logic of splitting an action into several smaller tasks, we can go a step further by specifying the output format.

In the following prompt, after explaining the steps involved, we'll ask the model to write specs in a specific format. It's just the beginning of specs, there's still a bit of "human" work to do.

In [16]:
prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following review delimited by 
  <> with 1 sentence.
2 - Identifies a possible solution to the user's problem.
3 - Write the user story to implement this evolution.

Use the following format:
Context: <business value>
User story: As a <persona>, I want <perform this action> so that <I can accomplish this goal>
Specs: <technical tasks to implement the evolution in the iOS application>
Acceptance criteria: Given <how things begin>, when <I take an action>, then <outcome of the action> in a table

Text: <{review_list[4]}>
"""
response = get_completion(prompt)
print(response)

1 - The reviewer expresses dissatisfaction with Airbnb, stating that hosts are allowed to do whatever they want with no consequences and that Airbnb erased their negative review.
2 - A possible solution to the user's problem could be for Airbnb to implement stricter policies and consequences for hosts who engage in fraudulent or unethical behavior, as well as ensuring that guest reviews are not arbitrarily erased.
3 - User story: As a guest, I want Airbnb to enforce stricter policies and consequences for hosts and ensure that guest reviews are not arbitrarily erased, so that I can have a safer and more reliable rental experience.
Specs: 
- Implement a system to monitor and investigate host behavior, including instances of demanding money and personal information through third-party websites.
- Establish clear guidelines and consequences for hosts who engage in fraudulent or unethical behavior.
- Improve the review system to prevent the arbitrary erasure of guest reviews.
- Provide a pl

#### 3.2.3. Reduce hallucinations 

A well-known problem with generative AI is that it can invent far-fetched stories with great confidence. One way to avoid this problem is to explicitly ask the model to search for relevant information, then analyze it.

After having explored reviews unitarily, let's imagine that we want to list all the pain points expressed by users and prioritize them. We'll explicitly ask the model to use information shared by users.

In [17]:
prompt = f"""
Your task is to help a product manager identify 
pain points for a product based on user reviews.

The list of bugs is intended for product developers, 
so should be technical in nature and focus on the 
technical improvements.

List the bugs described by users
using the reviews delimited by triple backticks.

First find relevant information from the reviews,
Then answer the question based on the relevant information.
Share only your answer.

Format everything as HTML that can be used in a website. 
Place the pain points in a table.

The table should have three columns.
In the first column include the bug description. 
In the second column include the way to solve it.
In the third column include the priority low, medium, or high.

Reviews: ```{review_list}```
"""
response = get_completion(prompt)
display(HTML(response))

Bug Description,Way to Solve,Priority
The app is missing the ability to manually check a guest out of a unit,Add a feature to allow hosts to manually check out guests and leave a review,Medium
"Notifications for ""_____ is arriving in 12 days"" cannot be turned off",Add an option to disable notifications for upcoming guest arrivals,Low
Shortcode messages display incorrect information if guest profile data is missing,Implement IF functions for shortcode messages to handle missing guest profile data,High
Poor quality search engine and hidden cleaning fees,Improve the search engine functionality and display cleaning fees upfront,High
Unsatisfactory cleanliness and maintenance of rental property,Ensure proper cleaning and maintenance of rental properties,Medium
Booking cancellation by host and unhelpful customer service,Provide better support for customers in case of booking cancellations,High
Lack of consequences for hosts and erasing relevant reviews,Enforce consequences for hosts and preserve relevant reviews,Medium
Age discrimination for users under 25,Remove age restrictions and allow users under 25 to book entire places,Low
Lack of safety measures and unhelpful customer support,Improve safety measures and provide better customer support,High
Inaccurate description and lack of assistance for fraudulent experiences,Enhance validation process for experiences and provide better assistance for fraudulent cases,Medium


**Note:** the previous prompt is particularly long, but I didn't write it on the first try. I started with a brief prompt and then iterated until I had a satisfactory and usable result.

The perfect prompt doesn't exist. **What does exist is a process of iteration to generate it.**

### 3.3. Summarize

We'll use the model to get an overview of each review.

To make sure that the model doesn't summarize in just any way, we'll specify that it's intended for the product team in charge of improving the product, and we'll give it a word limit.

In [18]:
# Loop through each review in the reviews variable
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \
    review from an app store to give feedback to the \
    product team, responsible for improving the \
    product.  

    Summarize the review below, delimited by triple 
    backticks, in at most 30 words, and focusing on any aspects \
    that are relevant to providing value to users. 

    Review: ```{review_list[i]}```
    """

    response = get_completion(prompt)
    print(i+1, response, "\n")

1 The review highlights the need for a manual check-out feature to leave reviews and change to new guests, the ability to turn off notifications, and the use of IF functions for shortcode messages. 

2 The review highlights poor search engine quality, stressful booking experiences, and unfair cancellation policies, suggesting the need for improvements in these areas to provide better value to users. 

3 Worst app for family vacation. Dirty, moldy, and unprofessional service. Lost $1200 and had to rent a hotel. Airbnb and host are only interested in taking your money. 

4 Disappointing customer service and lack of transparency in listing details led to a negative experience for the user. 

5 Airbnb's decline in quality is highlighted by a user's negative experience with a host demanding money and personal information through a third-party website. The user's review was deemed irrelevant by Airbnb, emphasizing the need for improved guest support and transparency. 

6 The reviewer express

We can also ask the model to extract information rather than summarize it.

In [19]:
# Loop through each review in the reviews variable
for i in range(len(reviews)):
    prompt = f"""
    Your task is to extract relevant issues \
    from a product review to give feedback to the \
    product team, responsible for improving the \
    product.  

    Summarize the review below, delimited by triple 
    backticks, in at most 30 words, and focusing on any aspects \
    that are relevant to providing value to users. 

    Review: ```{review_list[i]}```
    """

    response = get_completion(prompt)
    print(i+1, response, "\n")

1 The review highlights the need for a manual check-out feature to leave reviews and switch to new guests, the ability to turn off arrival notifications, and the option to use IF functions for shortcode messages to avoid errors and improve communication with guests. 

2 The review highlights poor search engine quality, hosts with outdated schedules, and a broken cancellation policy. It also mentions hidden cleaning fees and suggests finding an alternative booking app. 

3 The review highlights issues with cleanliness, including dust, dirty floors, and a greasy kitchen. There are also complaints about a broken curtain, hair in the beds, and mold in the bathroom. The lack of refund and unprofessionalism from both the host and Airbnb are mentioned, leading to a ruined vacation and additional expenses. 

4 The review highlights a lack of transparency in the booking process, resulting in unexpected costs and a disappointing customer service experience. The customer suggests that Airbnb shou

### 3.4. Infer

We'll now ask the model to infer and extract information from reviews.

A very useful analysis is to know what the sentiment of a review is. Let's ask the model.

In [20]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive"
or "negative".

Review text: '''{review_list[5]}'''
"""
response = get_completion(prompt)
print(response)

negative


Let's be more demanding and ask the model to extract a list of emotions.

In [21]:
prompt = f"""
Identify a list of emotions that the writer of the
following review is expressing. Include no more than
five items in the list. Format your answer as a list of
lower-case words separated by commas.

Review text: '''{review_list[5]}'''
"""
response = get_completion(prompt)
print(response)

discrimination, inconvenience, unhappy, angry, frustrated


Let's also ask the model to extract any problems that may have led to these emotions.

In [22]:
prompt = f"""
Determine five issues that are being discussed in the \
following product review, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Product review: '''{review_list[5]}'''
"""
response = get_completion(prompt)
print(response)

age discrimination, restricted account, customer service, kitchen access, alternative apps


Now, let's ask the model to extract the features and problems faced by users. We'll ask it to structure it answer.

In [23]:
prompt = f"""
Identify the following items from the review text: 
- Feature used by reviewer in the application
- The main issue that prompted the user to write the review

The review is delimited with triple backticks.
Format your response as a JSON object with
"Feature" and "Issue" as the keys. 
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.
  
Review text: '''{review_list[0]}'''
"""
response = get_completion(prompt)
print(response)

{
  "Feature": "Manually check out guests",
  "Issue": "Inability to manually check out guests and leave a review"
}


**Note:** We've completed these analyses on specific reviews. Just imagine applying them to thousands of reviews. Once consolidated, this data could quickly tell us which pain points to prioritize and which parts of the application.

### 3.5. Expand

Expand consists of asking the model to take a piece of text and generate a longer text.

This can be generate ideas as seen in the prompts above.

Here, we're going to ask the model to generate a response to a disgruntled user.

In [24]:
prompt = f"""
You are a customer service AI assistant.
Your task is to reply to a valued user.
Given the user review delimited by ```,
Generate a reply to thank the user for their review.
If the sentiment is positive or neutral, thank them for
their review.
If the sentiment is negative, apologize and suggest that
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
User review: ```{review_list[6]}```
"""
response = get_completion(prompt)
print(response)

Dear valued user,

Thank you for taking the time to share your experience with us. We sincerely apologize for the inconvenience and safety concerns you encountered during your stay. This is not the level of service we strive to provide.

We understand the importance of safety and we deeply regret that we were unable to address the issues you faced promptly. We apologize for any frustration or inconvenience this may have caused you.

We would like to assure you that your feedback is important to us and we take it seriously. We encourage you to reach out to our customer service team directly so that we can further investigate and assist you with resolving this matter. Our team is available 24/7 and is dedicated to providing the support you need.

Once again, we apologize for the negative experience you had and we appreciate your feedback. We hope to have the opportunity to make it right for you in the future.

Best regards,
AI customer agent


## 4. Talk to your AI user

Let's get down to business. Now, we're going to create a custom chatbot that has taken the reviews as input and will simulate a conversation with a fictitious user.

In addition to the previous helper function, we'll define get_completion_from_messages, a function to get a completion from a list of messages.

In [25]:
# Function to get a completion from a list of messages
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

Then, define collect_messages, a function to collect messages from the user and the assistant.

The goal of this function is to take all the messages from a conversation, put them in a list, and use this list as input for the helper function above, which will then generate the following response.

In [26]:
# Function to collect messages from the user and the assistant
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('Product Manager:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('AI User:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

We're going to define the initial context for this custom chatbot. In particular, we'll insist that this fictitious user not only gives his opinion, but does so honestly, even suggesting new ideas.

In [27]:
# Define a list called context to store the chat history
context = [ {'role':'system', 'content':f"""

You are a user of an application.

Your role is to respond to suggestions for improvement from 
the product team.

First, you take the suggestion and analyze it in your own use.

You answer whether or not it meets your needs, explaining why.

Then, you develop alternatives and bounce back with
relevant ideas.

Be sure to provide details that will help the product team 
improve the application.

Be honest, straight to the point, and not afraid to give
your sincere opinion.

Respond in a short, very conversational friendly style.

For context, here are reviews from previous users of the app:
```{review_list}```

"""} ]

Next, we'll write a few lines to build the interface.

In [28]:
# Import the panel library, a framework for creating interactive web apps
import panel as pn

In [29]:
# Load the panel extension to enable the use of widgets and other features
pn.extension()

In [30]:
# Create an empty list to store the widgets that disply the messages in a chat interface
panels = []

In [31]:
# Create a text input widget
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')

# Create a button widget
button_conversation = pn.widgets.Button(name="Submit Idea")

# Create an interactive object
# It calls collect_messages function whenever the button_conversation widget is clicked
interactive_conversation = pn.bind(collect_messages, button_conversation)

In [32]:
# Create a column panel
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

In [33]:
# Display the dashboard panel
dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Submit Idea')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

Try the following ideas:
- Add a feature to allow hosts to manually check out guests and leave a review.
- Add the ability to switch languages and translate a conversation.
- Improve the search engine functionality.
- Hide the cleaning costs well, and even double them to catch everyone off guard.
- Ensure proper cleaning and maintenance of rental properties.
- Remove support for customers in case of booking cancellations.